In [2]:
import pandas as pd
import numpy as np
from scipy.stats import chi2,norm

In [3]:
def stratified_sampling(df, column_name, num_samples):
    return df.groupby(column_name, group_keys=False).apply(lambda x: x.sample(n = num_samples, replace=False))

In [4]:
data = pd.read_csv("dataset\heart_2022_cleaned.csv")

In [5]:
def do_chi2_test(data,column1,column2,alpha = 0.01):
    # print("***************************************************************************************************************************")
    # print("Chi2 Test for ",column1 ," and " ,column2)
    observed={}
    cols=data[column1].unique()
    for i in range(len(data[column2])):
        if data[column2].iloc[i] not in observed:
            observed[data[column2].iloc[i]]={}
            for j in cols:
                observed[data[column2].iloc[i]][j]=0
        observed[data[column2].iloc[i]][data[column1].iloc[i]]+=1

    observed=pd.DataFrame(observed).T
    observed=observed.sort_index()
    observed=observed[sorted(observed.columns)]
    # print("Observed")
    # print(observed)
    # print()
    observed=observed.to_numpy()
    row_sums = observed.sum(axis=1)
    col_sums = observed.sum(axis=0)
    total = row_sums.sum()
    # expected = np.outer(row_sums, col_sums) / total

    expected=np.zeros(observed.shape)
    for i in range(observed.shape[0]):
        for j in range(observed.shape[1]):
            expected[i][j]=row_sums[i]*col_sums[j]/total
    exp=pd.DataFrame(expected)
    exp.columns=sorted(data[column1].unique())
    exp.index=sorted(data[column2].unique())
    # print("Expected") 
    # print(exp)
    chi2_stat = ((observed - expected)**2 / expected).sum()
   
    dof=(len(row_sums)-1)*(len(col_sums)-1)
    critical = chi2.ppf(1-alpha, dof)
    # print("chi2_stat: ",chi2_stat)
    # print("critical: ",critical)
    if chi2_stat>=critical:
        # print("Dependent (reject H0)")
        return [chi2_stat,critical,1]
    else:
        # print("Independent (fail to reject H0)")
        return [chi2_stat,critical,-1]

In [6]:
categorical_columns = data.select_dtypes(include=['object']).columns
categorical_columns

Index(['State', 'Sex', 'GeneralHealth', 'LastCheckupTime',
       'PhysicalActivities', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'AlcoholDrinkers', 'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver',
       'TetanusLast10Tdap', 'HighRiskLastYear', 'CovidPos'],
      dtype='object')

### $\chi^2$ Test for independence between age category and Heart Health ###

In [ ]:
import os

os.makedirs('output/stratified_sampling', exist_ok=True)

In [24]:
for col in categorical_columns:
    if col=='GeneralHealth':
        continue
    # for num_samples in range(1000,5001,1000):
    for num_samples in range(1,6,1):
        # file_path = os.path.join("output", "stratified_sampling", col, f"chi2_test_result_{num_samples}.txt")
        # f=open(file_path,"w")   
        f=open("output/stratified_sampling","w")
        for iteration in range(1,6):
            sample = stratified_sampling(data,'GeneralHealth',num_samples)
            f.write(f"<Iteration-{iteration}>\n")
            chi2_stat,chi2_critical,result = do_chi2_test(sample,col,'GeneralHealth',0.01)
            f.write(f"{chi2_stat}\n")
            f.write(f"{chi2_critical}\n")
            f.write(f"{result}\n")
        f.close()

PermissionError: [Errno 13] Permission denied: 'output/stratified_sampling'